In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

model_checkpoint = 'ufal/robeczech-base'
BATCH_SIZE = 32
transformers.utils.logging.set_verbosity_error()

### BABE train_test split (SKIP IF DONE)

In [ ]:
babe = load_dataset("csv", data_files=PATH + '/data/CS/raw/BABE/SG2.csv')['train']

In [ ]:
babe = babe.train_test_split(0.15,seed=42)

In [54]:
babe['train'].to_csv(PATH + '/data/CS/processed/BABE/train.csv',index=False)
babe['test'].to_csv(PATH + '/data/CS/processed/BABE/test.csv',index=False) #THIS IS FOR THE FINAL MODEL SELECTED,TUNED

125614

## Load data

In [2]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
data

Using custom data configuration default-84520668e41f8432
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-84520668e41f8432/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


Dataset({
    features: ['sentence', 'label'],
    num_rows: 3122
})

In [3]:
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [4]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [37]:
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    #per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=100,
    disable_tqdm = False,
    warmup_steps=10,
    save_total_limit=2,
    #load_best_model_at_end=True,
    #evaluation_strategy="steps",
    #metric_for_best_model = 'f1',)
    weight_decay=0.1,
    output_dir = './',
    learning_rate=4e-5)

## BEST PARAMS

PyTorch: setting up devices


In [ ]:
model_scores = {}

for model_name in config_data['models']:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                          tokenizer=tokenizer)
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores


Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-84520668e41f8432/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-f89f73809237ea4b.arrow
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/torch_formatter.py:44: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before conv

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 237
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


KeyboardInterrupt: 

***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 237
***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
***** Running training *****
  Num examples = 2497
  Num Epochs = 3
  Instantaneous batch size per device = 32
***** Running training *****
  Num examples = 2497
  Num Epochs = 3
***** Running training *****
  Num examples = 2497
***** Running training *****


In [79]:
model_scores['ufal/robeczech-base']

[0.7968, 0.776, 0.7868589743589743, 0.7788461538461539, 0.7612179487179487]

## WandB

In [12]:
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    #per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=10,
    disable_tqdm = False,
    warmup_steps=10,
    save_total_limit=2,
    #load_best_model_at_end=True,
    #evaluation_strategy="steps",
    #metric_for_best_model = 'f1',)
    weight_decay=0.1,
    output_dir = './',
    learning_rate=4e-5)

## BEST PARAMS

PyTorch: setting up devices


In [13]:
scores = []
model_name = 'fav-kky/FERNET-C5'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True);
data_collator = DataCollatorWithPadding(tokenizer=tokenizer);

token_full = preprocess_data(data,tokenizer,'sentence');

print("Running 5-fold CV on model: ",model_name,"...")
for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])

    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.to(device);
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
    tokenizer=tokenizer);
    trainer.train();
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
    break


loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/vocab.txt from cache at /home/horyctom/.cache/huggingface/

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/pytorch_model.bin from cache at /home/horyctom/.ca

Step,Training Loss
10,0.715600
20,0.630700
30,0.548900
40,0.626800
50,0.584300
60,0.534100
70,0.517300
80,0.565600
90,0.398600
100,0.393800




Training completed. Do not forget to share your model on huggingface.co/models =)




train/epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇██
train/learning_rate,██▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁
train/loss,█▇▆▇▆▆▆▆▄▄▃▄▄▄▄▃▂▂▂▁▃▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,3.0
train/global_step,237


In [9]:
np.mean(scores)

0.7741779487179486